copy from https://github.com/openai/openai-cookbook/blob/main/examples/Tag_caption_images_with_GPT4V.ipynb

In [1]:
!pip install --upgrade openai

DEPRECATION: textract 1.6.5 has a non-standard dependency specifier extract-msg<=0.29.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of textract or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 23.3.1 -> 24.1
[notice] To update, run: pip install --upgrade pip


In [4]:
from openai import OpenAI
import os

from IPython.display import Image, display
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np


In [3]:
client = OpenAI(api_key="")

In [ ]:
!wget https://raw.githubusercontent.com/RowanCompany/2024_AI_workshp/main/material/data/amazon_furniture_dataset.csv -O amazon_furniture_dataset.csv

In [5]:
dataset_path =  "amazon_furniture_dataset.csv"
df = pd.read_csv(dataset_path)
df.head()

,asin,url,title,brand,price,availability,categories,primary_image,images,upc,...,color,material,style,important_information,product_overview,about_item,description,specifications,uniq_id,scraped_at
0,B0CJHKVG6P,https://www.amazon.com/dp/B0CJHKVG6P,"GOYMFK 1pc Free Standing Shoe Rack, Multi-laye...",GOYMFK,$24.99,Only 13 left in stock - order soon.,"['Home & Kitchen', 'Storage & Organization', '...",https://m.media-amazon.com/images/I/416WaLx10j...,['https://m.media-amazon.com/images/I/416WaLx1...,NaN,...,White,Metal,Modern,[],"[{'Brand': ' GOYMFK '}, {'Color': ' White '}, ...",['Multiple layers: Provides ample storage spac...,"multiple shoes, coats, hats, and other items E...","['Brand: GOYMFK', 'Color: White', 'Material: M...",02593e81-5c09-5069-8516-b0b29f439ded,2024-02-02 15:15:08
1,B0B66QHB23,https://www.amazon.com/dp/B0B66QHB23,"subrtex Leather ding Room, Dining Chairs Set o...",subrtex,NaN,NaN,"['Home & Kitchen', 'Furniture', 'Dining Room F...",https://m.media-amazon.com/images/I/31SejUEWY7...,['https://m.media-amazon.com/images/I/31SejUEW...,NaN,...,Black,Sponge,Black Rubber Wood,[],NaN,['【Easy Assembly】: Set of 2 dining room chairs...,subrtex Dining chairs Set of 2,"['Brand: subrtex', 'Color: Black', 'Product Di...",5938d217-b8c5-5d3e-b1cf-e28e340f292e,2024-02-02 15:15:09
2,B0BXRTWLYK,https://www.amazon.com/dp/B0BXRTWLYK,Plant Repotting Mat MUYETOL Waterproof Transpl...,MUYETOL,$5.98,In Stock,"['Patio, Lawn & Garden', 'Outdoor Décor', 'Doo...",https://m.media-amazon.com/images/I/41RgefVq70...,['https://m.media-amazon.com/images/I/41RgefVq...,NaN,...,Green,Polyethylene,Modern,[],"[{'Brand': ' MUYETOL '}, {'Size': ' 26.8*26.8 ...","['PLANT REPOTTING MAT SIZE: 26.8"" x 26.8"", squ...",NaN,"['Brand: MUYETOL', 'Size: 26.8*26.8', 'Item We...",b2ede786-3f51-5a45-9a5b-bcf856958cd8,2024-02-02 15:15:09
3,B0C1MRB2M8,https://www.amazon.com/dp/B0C1MRB2M8,"Pickleball Doormat, Welcome Doormat Absorbent ...",VEWETOL,$13.99,Only 10 left in stock - order soon.,"['Patio, Lawn & Garden', 'Outdoor Décor', 'Doo...",https://m.media-amazon.com/images/I/61vz1Igler...,['https://m.media-amazon.com/images/I/61vz1Igl...,NaN,...,A5589,Rubber,Modern,[],"[{'Brand': ' VEWETOL '}, {'Size': ' 16*24INCH ...","['Specifications: 16x24 Inch ', "" High-Quality...",The decorative doormat features a subtle textu...,"['Brand: VEWETOL', 'Size: 16*24INCH', 'Materia...",8fd9377b-cfa6-5f10-835c-6b8eca2816b5,2024-02-02 15:15:10
4,B0CG1N9QRC,https://www.amazon.com/dp/B0CG1N9QRC,JOIN IRON Foldable TV Trays for Eating Set of ...,JOIN IRON Store,$89.99,Usually ships within 5 to 6 weeks,"['Home & Kitchen', 'Furniture', 'Game & Recrea...",https://m.media-amazon.com/images/I/41p4d4VJnN...,['https://m.media-amazon.com/images/I/41p4d4VJ...,NaN,...,Grey Set of 4,Iron,X Classic Style,[],NaN,['Includes 4 Folding Tv Tray Tables And one Co...,Set of Four Folding Trays With Matching Storag...,"['Brand: JOIN IRON', 'Shape: Rectangular', 'In...",bdc9aa30-9439-50dc-8e89-213ea211d66a,2024-02-02 15:15:11


# 키워드 달기

In [6]:
system_prompt = '''
    You are an agent specialized in tagging images of furniture items, decorative items, or furnishings with relevant keywords that could be used to search for these items on a marketplace.
    
    You will be provided with an image and the title of the item that is depicted in the image, and your goal is to extract keywords for only the item specified. 
    
    Keywords should be concise and in lower case. 
    
    Keywords can describe things like:
    - Item type e.g. 'sofa bed', 'chair', 'desk', 'plant'
    - Item material e.g. 'wood', 'metal', 'fabric'
    - Item style e.g. 'scandinavian', 'vintage', 'industrial'
    - Item color e.g. 'red', 'blue', 'white'
    
    Only deduce material, style or color keywords when it is obvious that they make the item depicted in the image stand out.

    Return keywords in the format of an array of strings, like this:
    ['desk', 'industrial', 'metal']
    
'''

def analyze_image(img_url, title):
    response = client.chat.completions.create(
    model="gpt-4-vision-preview",
    messages=[
        {
            "role": "system",
            "content": system_prompt
        },
        {
            "role": "user",
            "content": [
                {
                    "type": "image_url",
                    "image_url": img_url,
                },
            ],
        },
        {
            "role": "user",
            "content": title
        }
    ],
        max_tokens=300,
        top_p=0.1
    )

    return response.choices[0].message.content

In [7]:
examples = df.iloc[:5]

In [8]:
for index, ex in examples.iterrows():
    url = ex['primary_image']
    img = Image(url=url)
    display(img)
    result = analyze_image(url, ex['title'])
    print(result)
    print("\n\n")

['shoe rack', 'free standing', 'multi-layer', 'metal', 'with hooks', 'white']





['dining chairs', 'set of 2', 'leather', 'black']





['plant repotting mat', 'waterproof', 'portable', 'foldable', 'easy to clean', 'green']





['doormat', 'absorbent', 'non-slip', 'brown', 'coir']





['tv tray table set', 'foldable', 'iron', 'grey']





# 설명 달기

In [10]:
describe_system_prompt = '''
    You are a system generating descriptions for furniture items, decorative items, or furnishings on an e-commerce website.
    Provided with an image and a title, you will describe the main item that you see in the image, giving details but staying concise.
    You can describe unambiguously what the item is and its material, color, and style if clearly identifiable.
    If there are multiple items depicted, refer to the title to understand which item you should describe.
    '''

def describe_image(img_url, title):
    response = client.chat.completions.create(
    model="gpt-4-vision-preview",
    temperature=0.2,
    messages=[
        {
            "role": "system",
            "content": describe_system_prompt
        },
        {
            "role": "user",
            "content": [
                {
                    "type": "image_url",
                    "image_url": img_url,
                },
            ],
        },
        {
            "role": "user",
            "content": title
        }
    ],
    max_tokens=300,
    )

    return response.choices[0].message.content

In [11]:
for index, row in examples.iterrows():
    print(f"{row['title'][:50]}{'...' if len(row['title']) > 50 else ''} - {row['url']} :\n")    
    url = row['primary_image']
    img = Image(url=url)
    display(img)
    img_description = describe_image(row['primary_image'], row['title'])
    print(img_description)
    print("\n\n")

GOYMFK 1pc Free Standing Shoe Rack, Multi-layer Me... - https://www.amazon.com/dp/B0CJHKVG6P :



This is a multi-functional free-standing shoe rack featuring a metal construction with a white finish. It is designed with multiple layers to accommodate various pairs of shoes across its shelves. The rack includes 8 double hooks arranged along its upper section, providing additional storage for hanging items such as hats, scarves, and bags. Its vertical design makes it suitable for placement in living rooms, bathrooms, hallways, or other areas where space optimization is desired. The clean lines and simple color scheme allow it to blend seamlessly with a variety of interior decor styles.



subrtex Leather ding Room, Dining Chairs Set of 2,... - https://www.amazon.com/dp/B0B66QHB23 :



This image features a set of two black dining chairs. The chairs are upholstered in a leather-like material, providing a sleek and modern look. The design includes a high backrest with subtle stitching details that create vertical lines, adding an elegant touch to the overall minimalist aesthetic. The legs of the chairs are straight and also finished in black, which complements the upholstery and enhances the contemporary style. These chairs would make a sophisticated addition to any dining room setting.



Plant Repotting Mat MUYETOL Waterproof Transplanti... - https://www.amazon.com/dp/B0BXRTWLYK :



This image shows a square-shaped plant repotting mat designed for indoor gardening tasks such as transplanting or changing the soil of plants. The mat is a vibrant green color and appears to be made from a waterproof material, which makes it easy to clean. It measures 26.8 inches by 26.8 inches, providing ample space for gardening activities.

The mat features raised corners with snap buttons on each corner, allowing it to fold up into a tray-like shape to contain soil and debris. This design helps to keep the work area tidy and prevents spillage. The mat is portable and can be easily folded for storage or transport, making it a convenient accessory for gardeners.

In the image, the mat is shown with some small potted succulent plants and a few gardening tools, illustrating its intended use for tasks like repotting and transplanting small plants. These additional items are not included with the mat but serve to demonstrate its functionality. The mat itself is a practical item for anyon

This is a rectangular doormat featuring a playful design, ideal for pickleball enthusiasts. The mat has a natural coir fiber surface, known for its durability and excellent scraping properties to remove dirt and debris from shoes. The text "it's a good day to play PICKLEBALL" is printed in bold, black letters, creating a welcoming and sporty vibe. Below the text, there's an illustration of two pickleball paddles crossed over a ball, adding to the theme. The mat measures approximately 16x24 inches, making it a suitable size for most doorways. It is designed to be absorbent and features a non-slip backing to ensure safety and prevent movement on various floor types. This doormat would be a functional and decorative addition to the entrance of a home, especially for those who enjoy the sport of pickleball.



JOIN IRON Foldable TV Trays for Eating Set of 4 wi... - https://www.amazon.com/dp/B0CG1N9QRC :



This image features a set of two foldable TV trays, which are part of a set of four. Each tray has a sleek, rectangular surface with a wood grain finish in a grey tone, providing a modern and neutral look that can complement various interior decor styles. The trays are supported by a sturdy, black iron frame with an X-shaped base that allows for easy folding and storage. The design is practical and space-saving, ideal for use in small living spaces for activities such as eating, working, or as a temporary side table. The set includes a matching stand for convenient storage of all four trays when not in use, ensuring they can be neatly tucked away.



